In [ ]:
!pip install transformers tqdm

In [ ]:
!HF_ENDPOINT=https://hf-mirror.com huggingface-cli download bert-base-chinese

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np

# 加载文件

In [ ]:
with open('文本.txt', 'r', encoding='utf-8') as file:
  sentences = file.readlines()
print('文本条数: ', len(sentences))
print('预览第一条: ', sentences[0])

# 训练词向量

In [ ]:
# 加载预训练模型和tokenizer
# 模型名字直接写入bert-base-chinese这个简化模型名就可以了，https://huggingface.co/google-bert/bert-base-chinese
# 如果无法用梯子的话，可以本地下载：huggingface-cli download --resume-download bert-base-chinese
model_name = "bert-base-chinese"
# model_name = "hfl/chinese-bert-wwm"

# 也可以试试使用哈工大的模型，model_name = "hfl/chinese-bert-wwm"
# 注意提前需要下载huggingface-cli download --resume-download hfl/chinese-bert-wwm

# 加载模型
# 会从huggingface中下载模型
# 源码：class PreTrainedModel(nn.Module....)
# 所以，这里创建的既是PreTrainedModel类的实例，也是torch.nn.Module的实例
# 对于警告Some weights of the model checkpoint，对与我们的任务，可以不用在意
# 相关讨论：https://blog.csdn.net/PolarisRisingWar/article/details/123974645   https://huggingface.co/google-bert/bert-base-uncased/discussions/4
model = BertModel.from_pretrained(model_name)

# 加载tokenizer
# 使用Tokenizer，就是为了将输入的句子加工为bert模型可以处理的格式
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
# 将模型放置在GPU上
# torch.cuda.is_available()，检测cuda是否可用
# torch.device()设置张量运算在哪个设备上进行
# device = torch.device("cpu")，表示在CPU进行
# device = torch.device("cuda")，表示在GPU进行
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 把模型放到cpu或gpu
model.to(device)
# 将模型设置为评估模式，https://blog.csdn.net/weixin_45275599/article/details/131524189
model.eval() 

# 切分数据
batch_size = 16  # 批大小
data_loader = DataLoader(sentences, batch_size=batch_size)

In [ ]:
# ---- 文本转向量 ----
# 生成的向量存放在这里
cls_embeddings = []

# 使用tqdm显示处理进度
# tqdm b站教程：https://www.bilibili.com/video/BV1ZG411M7Ge/?spm_id_from=333.337.search-card.all.click&vd_source=eace37b0970f8d3d597d32f39dec89d8
for batch_sentences in tqdm(data_loader):
    # tokenizer官方文档：https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__
    # truncation=True，对输入句子进行截断，这里确保最大长度不超过512个字
    # max_length：不设置的话，默认会截断到该模型可接受的最大长度
    # padding=True 或 padding='longest': 将所有句子填充到批次中最长句子的长度
    # padding="max_length": 将所有句子填充到由 max_length 参数指定的长度
    inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
    # print(123, inputs.input_ids[0], tokenizer.decode(inputs.input_ids[0]))
    
    # 把编码好的数据，也放在device上，It is necessary to have both the model, and the data on the same device, either CPU or GPU
    # https://huggingface.co/docs/transformers/v4.39.2/en/main_classes/tokenizer#transformers.BatchEncoding.to
    # https://stackoverflow.com/questions/63061779/pytorch-when-do-i-need-to-use-todevice-on-a-model-or-tensor
    inputs.to(device)

    # 设置不要计算梯度
    # 一般来说，如果我们只是用模型进行“预测”，而不涉及对模型进行更新时，就不需要计算梯度，以此来节约内存，增加运算效率
    # with上下文中，对model的调用将遵循torch.no_grad()，即不会计算梯度
    with torch.no_grad():
        outputs = model(**inputs)

    # 把这一批词向量存入cls_embeddings容器中
    # tensor.cpu() 将张量移动到 CPU
    # tensor.numpy() 将 CPU 上的张量转换为 NumPy 数组
    cls_embeddings.append(outputs.last_hidden_state[:, 0].cpu().numpy()) # 只取CLS对应的向量

# 合并句子向量
cls_embeddings = np.vstack(cls_embeddings)
print('最终生成的词向量', type(cls_embeddings), cls_embeddings.shape)

# ---- 保存词嵌入向量 ----
# 保存句子向量到npy文件
# 官方文档：https://numpy.org/doc/stable/reference/generated/numpy.save.html
output_file = "emb.npy"
np.save(output_file, cls_embeddings)
print("词向量存储于: ", output_file)

embeddings = np.load(output_file)
print("加载回来，验证一下：", type(embeddings), embeddings.shape)